In [ ]:
import numpy as np

import tensorflow as tf
import tensorflow.keras.optimizers as optimizers

from model import *
from input import NetworkInput
from config import *


from data import createFeaturesDescription

from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard


from datetime import datetime

import os
import sys

### Configuration

In [ ]:
batch_size=512
num_steps=32

keep_prob=1
keep_prob_recurrent=1
keep_prob_dense_layer=1

data_augmentation=True
name_train='train'

hidden_size=128
num_layers=2

stride = 2

run_with_papermill=False #if true, verbose mode will be set to 2 (1 line/epoch)

kernel_regularizer_l2=0
recurrent_regularizer_l2=0

#TODO use_HPRMS
use_F0=False
F0_binary_values=False

use_deltas=False

nb_epochs=30

train_model=True
save_weights=True


load_weights=False
weights_filename="./logdir/2020-10-29_17-50-58-size_3_128/weights/weights_2020-10-29_17-50-58.h5" 

In [ ]:
# Parameters
hidden_size = 180
use_F0 = True
F0_binary_values = True
use_deltas = True
keep_prob = 0.8
keep_prob_dense_layer = 0.8
run_with_papermill = True
name_train = "2_180_w_deltas_dropout_20_w_F0"


In [ ]:

config=Config(batch_size, num_steps, keep_prob=keep_prob, keep_prob_recurrent=keep_prob_recurrent, 
              hidden_size=hidden_size, num_layers=num_layers, 
              kernel_regularizer_l2=kernel_regularizer_l2, recurrent_regularizer_l2=recurrent_regularizer_l2)
config=completedConfig(config) #take default params for unspecified params


In [ ]:
config

In [ ]:

features_description=createFeaturesDescription(F0=use_F0) #Features RMS, RMS HP


### Input

In [ ]:
languages = ["Danish", "Dutch", "English", "Finnish",
    "French", "German", "Hungarian", "Italian",
    "Japanese", "Korean", "Mandarin", "Polish",
    "Portuguese", "Russian", "Spanish",
    "Swedish", "Turkish", "Estonian", "Arabic", "Czech", "Romanian",
    "Basque", "Catalan"]

#Remove languages with not enough data
languages.remove("Czech")
languages.remove("Romanian")

# FIRST VERSION
#languages = ['Danish', 'Russian', 'Mandarin', 'Finnish', 'Dutch', 'English', 'Hungarian', 'Swedish', 
#             'Italian', 'French', 'Japanese', 'German', 'Portuguese', 'Polish', 'Spanish', 'Korean']

sets ={}

set_folds=[0]

sets_folds = {"train" : [0, 1, 2],
        "test":[3,4],
        "test1" : [3],
        "test2" : [4]}

set_name='train'
sets[set_name] = NetworkInput(config, folder='./Scores', 
                     subfolder=["fold_{}/".format(k_fold) for k_fold in sets_folds[set_name]],
            stride=stride, verbose=True,                                    
             languages=languages, name=set_name, features_description=features_description,
                              F0_binary_values=F0_binary_values,
                             data_augmentation=data_augmentation, use_deltas=use_deltas)

set_name='test'
sets[set_name] = NetworkInput(config, folder='./Scores', for_evaluation=True,
        subfolder=["fold_{}/".format(k_fold) for k_fold in sets_folds[set_name]],
            stride=stride, verbose=True,                                    
             languages=None, languages_model=languages, name=set_name,
                features_description=features_description, 
             F0_binary_values=F0_binary_values,
                use_deltas=use_deltas) #autodetect languages

In [ ]:
networkInput=sets["train"]
networkInputTest=sets["test"]

### Model

In [ ]:
model=build_model(config, networkInput)
model.summary()

In [ ]:
if load_weights:
    model.load_weights(weights_filename)

In [ ]:
if train_model:
    #SUMMARIES
    today=datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    logdir=f"./logdir/{today}-{name_train}"
    
    #summary_writer=tf.summary.create_file_writer(logdir)
    #summary_writer.set_as_default()
    
    #write_summaries_step=networkInput.nbr_batchs//20  #now every epoch
    #print(f'write summaries every {write_summaries_step} (non-splitted) batches')
    
    #OPTIM (NAG)
    true_nb_batches=networkInput.nbr_batchs*networkInput.num_slices_by_example
    true_nb_batches_test=networkInputTest.nbr_batchs*networkInputTest.num_slices_by_example
    lr_0=0.2  #start: 0.2
    lr_decay=0.93
    optim=optimizers.SGD(momentum=0.9, nesterov=True) #l_rate=0.001

    def lr_schedule(epoch):
        lr = lr_0*lr_decay**(epoch-1)
        tf.summary.scalar('learning rate', data=lr, step=epoch)
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)
    
    #CALLBACKS
    
    #simplePBar=Simple_progressBar_callback(networkInput, min_step=0.05)

    tensorboardCallback=TensorBoard(log_dir=logdir, histogram_freq=1, write_graph=True,
              update_freq='epoch', profile_batch=2)  
    #before:  update_freq=write_summaries_step*networkInput.num_slices_by_example,
    forgetStates=Forget_states_callback(networkInput, model, verbose=False)
    callbacksList=[forgetStates,lr_callback, tensorboardCallback]
    
#METRICS
acc_slices=[AccuracyStateless(networkInput, ind_batch_compute=k) for k in range(networkInput.num_slices_by_example)]
#top3_slices=[TopKAccuracyStateless(networkInput, k=3, ind_batch_compute=j) for j in range(networkInput.num_slices_by_example)]

metricsList=[KL_divStateless(networkInput), crossEntropyStateless(networkInput)]
metricsList+=acc_slices
metricsList+=[TopKAccuracyStateless(networkInput, k=3)] #top3  end of seq
#metricsList+=top3_slices

KLLoss=tf.keras.losses.KLDivergence()

In [ ]:
if train_model:
    model.compile(optimizer=optim, loss=KLLoss, metrics=metricsList)
else:
    model.compile(loss=KLLoss, metrics=metricsList)

In [ ]:
verbose_mode= 2 if run_with_papermill else 1
    
if train_model:
    for epoch in range(nb_epochs):
        model.fit(networkInput.sliced_batch, steps_per_epoch=true_nb_batches, initial_epoch=epoch, epochs=epoch+1,
                  shuffle=False, verbose=verbose_mode, callbacks=callbacksList, validation_data=networkInputTest.sliced_batch, 
                 validation_steps=true_nb_batches_test)
else: #evaluate
    true_nb_batches=networkInputTest.nbr_batchs*networkInputTest.num_slices_by_example
    forgetStates=Forget_states_callback(networkInputTest, model, verbose=False)
    callbacksList=[forgetStates]
    model.evaluate(networkInputTest.sliced_batch, verbose=verbose_mode, steps=true_nb_batches,callbacks=callbacksList)

In [ ]:
#SAVE WEIGHTS
weights_dir=f"{logdir}/weights"
if save_weights:
    os.makedirs(weights_dir)
    model.save_weights(f"{weights_dir}/weights_{today}.h5")

#TODO
#SAVE MODEL

In [ ]:
if train_model: #save info on input/model
    orig_stdout = sys.stdout
    with open(f"{logdir}/info.txt", 'w') as f:
        sys.stdout = f
        print(name_train)
        print(networkInput)
        
        model.summary()
        print(config)
        
        print(networkInputTest)
    
    sys.stdout = orig_stdout